In [ ]:
# measure weights for mulitple epochs
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import train
import argparse

parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-s', '--seed',
    type=float,
    default=1,
    help="Seed for randomness",
    dest="seed"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)

args=parser.parse_args()
model_str=args.name
layer_str=args.layer
tf.random.set_seed(args.seed)

exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
model,weights=train.weight_measure(model,layer_str,10,final_learning_rate,x_train,y_train,batch_size)
np.save(model_str+'/weights',weights)

In [ ]:
# compute the principal components
import numpy as np
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
args=parser.parse_args()
model_str=args.name

weights=np.load(model_str+'/weights.npy')
Cov=np.cov(weights.T)
variance,pcomp_cov=np.linalg.eigh(Cov)
pcomp=pcomp_cov.T[::-1]
np.save(model_str+'/pc',pcomp)

In [ ]:
# compute accuracies and losses for different number of components added
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import analysis
import argparse
parser = argparse.ArgumentParser(
    prog='train MLP256',
    description='train MLP256',
    epilog='Based on the rmt package.')
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)
parser.add_argument(
    '-m', '--mode',
    type=str,
    default="evh",
    help="type of input vectors, evh or pc",
    dest="mode"
)
parser.add_argument(
    '-a', '--abs',
    type=bool,
    default=True,
    help="whether eigenvalues should be added in order of magnitude or sign",
    dest="abs"
)

args=parser.parse_args()
model_str=args.name
md_str=args.mode
layer_str=args.layer


exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
if md_str=="evh":
        if args.abs:
            sort_list=np.argsort(np.abs(np.load(model_str+'/ewh_'+layer_str+'.npy')))
            print(sort_list)
            print(np.take(np.load(model_str+'/ewh_'+layer_str+'.npy'),sort_list,axis=0)[::-1])
            vec=np.take(np.load(model_str+'/evh_'+layer_str+'.npy'),sort_list,axis=0)[::-1]
        else:
            vec=np.load(model_str+'/evh_'+layer_str+'.npy')
else:
    vec=np.load(model_str+'/'+md_str+'.npy')
accs=analysis.acc_components(model,layer_str,vec,x_train,y_train,x_test,y_test)
if args.abs:
    np.save(model_str+'/accabs_'+layer_str+md_str,accs)
else:
    np.save(model_str+'/acc_'+layer_str+md_str,accs)